# Getting HRA ASCT+B Table data


This notebook exemplarily retrieves the most recent versions of all ASCT+B tables in the latest hra collection in the JSON format, which is one of the most widely used data exchange formats, and that can be read by almost every modern programming language. It then selects the table for the kidney. It then inspects the kidney table by outputting the number of rows, and finally prints a small selection of AS, CT, gene Bs, and scientific references for a particular AS-CT-B record. 

In [6]:
# Install pre-requisite libraries
%pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Imports / functions


In [7]:
import requests
from pprint import pprint


def fetch_json(purl):
    response = requests.get(purl, headers={"Accept": "application/json"})
    response.raise_for_status()
    return response.json()


def is_asctb_table(purl):
    return (
        purl.startswith("https://purl.humanatlas.io/asct-b/")
        and "crosswalk" not in purl
    )


def get_latest_asctb_data():
    hra_collection = fetch_json("https://purl.humanatlas.io/collection/hra")
    digital_objects = hra_collection["metadata"]["had_member"]
    tables = {}
    for purl in sorted(filter(is_asctb_table, digital_objects)):
        table_name = purl.split("/")[-2].replace('-', '_')
        table_data = fetch_json(purl)
        table_rows = table_data["data"]["asctb_record"]
        tables[table_name] = table_rows
    return tables

## Get the latest HRA ASCT+B table data


In [8]:
tables = get_latest_asctb_data()
print("Fetched", len(tables), "ASCT+B tables")

Fetched 34 ASCT+B tables


## Inspect the Kidney table


In [9]:
kidney = tables["kidney"]
print("The kidney table has", len(kidney), "rows")

The kidney table has 78 rows


In [10]:

def format_term(str):
    return str.replace("https://purl.org/ccf/ASCTB-TEMP_", "ASCTB-TEMP:")


def simple_list(elts):
    return [f'{format_term(x["source_concept"])} ({x["ccf_pref_label"]})' for x in elts]


print("1st Row of Kidney Table")
record = kidney[0]

print("\nAnatomical Structures:")
pprint(simple_list(record["anatomical_structure_list"]))

print("\nCell Types:")
pprint(simple_list(record["cell_type_list"]))

print("\nGene Biomarkers:")
pprint(simple_list(record.get("gene_marker_list", [])))

print("\nReferences:")
pprint(record["reference_list"])

1st Row of Kidney Table

Anatomical Structures:
['UBERON:0002113 (kidney)', 'UBERON:0002015 (kidney capsule)']

Cell Types:
['CL:0000499 (stromal cell)',
 'ASCTB-TEMP:capsule-mesenchymal-stromal-cell (capsule mesenchymal stromal '
 'cell)']

Gene Biomarkers:
['HGNC:3802 (Foxd1)', 'HGNC:1975 (VSX2)']

References:
[{'doi': 'https://doi.org/10.1371/journal.pone.0187118',
  'external_id': 'https://doi.org/10.1371/journal.pone.0187118',
  'id': 'https://purl.humanatlas.io/asct-b/kidney/v1.7#R1-REF1',
  'notes': 'CD73, CD105, PDGFR? and ?-SMA',
  'order_number': 1,
  'record_number': 1,
  'type_of': ['ccf:ReferenceRecord']}]
